In [1]:
import numpy as np
import pandas as pd
import gc
import time
import pickle
import catboost as cat

In [2]:
df = pd.read_pickle('matrix_fe2.pkl')
df.drop(['date_shop_item_avg_item_cnt_lag_1_y','ID'], axis=1, inplace=True)

In [3]:
%%time
df[['delta_revenue_lag_1','item_cnt_month_lag_1','item_cnt_month_lag_2','item_cnt_month_lag_3','date_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_2','date_item_avg_item_cnt_lag_3','date_shop_avg_item_cnt_lag_1','date_shop_avg_item_cnt_lag_2','date_shop_avg_item_cnt_lag_3','date_cat_avg_item_cnt_lag_1','date_shop_cat_avg_item_cnt_lag_1','date_shop_item_avg_item_cnt_lag_1_x','date_shop_subtype_avg_item_cnt_lag_1','date_city_avg_item_cnt_lag_1','date_item_city_avg_item_cnt_lag_1','delta_price_lag']] = df[['delta_revenue_lag_1','item_cnt_month_lag_1','item_cnt_month_lag_2','item_cnt_month_lag_3','date_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_2','date_item_avg_item_cnt_lag_3','date_shop_avg_item_cnt_lag_1','date_shop_avg_item_cnt_lag_2','date_shop_avg_item_cnt_lag_3','date_cat_avg_item_cnt_lag_1','date_shop_cat_avg_item_cnt_lag_1','date_shop_item_avg_item_cnt_lag_1_x','date_shop_subtype_avg_item_cnt_lag_1','date_city_avg_item_cnt_lag_1','date_item_city_avg_item_cnt_lag_1','delta_price_lag']].astype(str)

CPU times: user 1min 51s, sys: 8.92 s, total: 2min
Wall time: 1min 47s


In [4]:

X_train = df[df.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = df[df.date_block_num < 33]['item_cnt_month']
X_valid = df[df.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = df[df.date_block_num == 33]['item_cnt_month']
X_test = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)
del df
gc.collect();

In [5]:
model = cat.CatBoostRegressor(task_type='GPU',verbose=100,
    depth=12, 
    loss_function='RMSE', 
    eval_metric='RMSE', 
    random_seed=42, 
    od_type='Iter', 
    od_wait=10,
    allow_writing_files=False)

In [6]:
%%time
#categorical_features_indices = cat_features= [0,1,2,3,4,5,6,7] #list(range(0, X_train.shape[1]))

categorical_features_indices = list(range(0, X_train.shape[1]))

CPU times: user 33 µs, sys: 4 µs, total: 37 µs
Wall time: 45.5 µs


In [7]:
%%time
train_pool = cat.Pool(X_train, Y_train, cat_features=categorical_features_indices)

CPU times: user 8min 37s, sys: 2.2 s, total: 8min 40s
Wall time: 8min 37s


In [8]:
%%time
validation_pool = cat.Pool(X_valid, Y_valid, cat_features=categorical_features_indices)

CPU times: user 13.2 s, sys: 47.8 ms, total: 13.2 s
Wall time: 13.2 s


In [9]:
del X_train,Y_train
gc.collect();

In [10]:
%%time
model.fit(train_pool, eval_set=validation_pool, use_best_model=True, verbose=True)

0:	learn: 1.2308225	test: 1.1533457	best: 1.1533457 (0)	total: 35.4s	remaining: 9h 49m 54s
1:	learn: 1.2116772	test: 1.1426382	best: 1.1426382 (1)	total: 1m 10s	remaining: 9h 50m 25s
2:	learn: 1.1933235	test: 1.1305235	best: 1.1305235 (2)	total: 1m 46s	remaining: 9h 48m 28s
3:	learn: 1.1757926	test: 1.1219568	best: 1.1219568 (3)	total: 2m 21s	remaining: 9h 48m 37s
4:	learn: 1.1588729	test: 1.1117206	best: 1.1117206 (4)	total: 2m 56s	remaining: 9h 46m 9s
5:	learn: 1.1427169	test: 1.1028054	best: 1.1028054 (5)	total: 3m 31s	remaining: 9h 45m 15s
6:	learn: 1.1271701	test: 1.0957204	best: 1.0957204 (6)	total: 4m 7s	remaining: 9h 44m 27s
7:	learn: 1.1124427	test: 1.0888848	best: 1.0888848 (7)	total: 4m 42s	remaining: 9h 43m 24s
8:	learn: 1.0982636	test: 1.0829534	best: 1.0829534 (8)	total: 5m 16s	remaining: 9h 41m 43s
9:	learn: 1.0846359	test: 1.0770993	best: 1.0770993 (9)	total: 5m 51s	remaining: 9h 40m 46s
10:	learn: 1.0716675	test: 1.0712705	best: 1.0712705 (10)	total: 6m 26s	remaining: 

In [11]:
%%time
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

In [12]:
Y_pred

array([0.4481245 , 0.13815697, 0.54128994, ..., 0.24127515, 0.11922956,
       0.36826589])

In [13]:
Y_test

array([1.14547921, 0.19348547, 1.26323208, ..., 0.32823852, 0.1137349 ,
       0.38517391])

In [ ]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [17]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('cat_submission1.csv', index=False)

In [16]:
import os
PATH_TO_DATA = 'data/predict_future_sales'
test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'))

In [ ]:
#Your submission scored 1.04083